<a href="https://colab.research.google.com/github/habrev/Rossmann-Pharmaceuticals/blob/task-2/notebooks/prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dependency

In [10]:
import pandas as pd
import matplotlib as plt
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dataset merging

# file imported

In [5]:
train = pd.read_csv('train.csv')
store = pd.read_csv('store.csv')

In [9]:
# Merge the datasets on the 'Store' column
merged_data = pd.merge(train, store, on='Store', how='left')

# Display the first few rows of the merged dataset
print(merged_data.head())

# Save the merged dataset to a CSV file (optional)
output_path = 'merged_data.csv'
merged_data.to_csv(output_path, index=False)
print(f"\nmerged file saved to {output_path}")

   Store  DayOfWeek        Date    Sales  Customers  Open  Promo StateHoliday  \
0      1          5  2015-07-31   5263.0      555.0   1.0    1.0            0   
1      2          5  2015-07-31   6064.0      625.0   1.0    1.0            0   
2      3          5  2015-07-31   8314.0      821.0   1.0    1.0            0   
3      4          5  2015-07-31  13995.0     1498.0   1.0    1.0            0   
4      5          5  2015-07-31   4822.0      559.0   1.0    1.0            0   

   SchoolHoliday StoreType Assortment  CompetitionDistance  \
0            1.0         c          a               1270.0   
1            1.0         a          a                570.0   
2            1.0         a          a              14130.0   
3            1.0         c          c                620.0   
4            1.0         a          a              29910.0   

   CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
0                        9.0                    2008.0       0   
1       

# Preprocessing

In [13]:
# Load the merged dataset
merged_data = pd.read_csv('merged_data.csv')


# Convert 'Date' to datetime, coercing errors
merged_data['Date'] = pd.to_datetime(merged_data['Date'], errors='coerce')

# Check for invalid dates and print a summary
invalid_dates = merged_data[merged_data['Date'].isnull()]
if not invalid_dates.empty:
    print(f"Invalid dates found and removed:\n{invalid_dates}")

# Remove rows with invalid or missing dates
merged_data.dropna(subset=['Date'], inplace=True)

# Step 1: Handle missing values
# Fill missing competition data with default values
merged_data['CompetitionDistance'].fillna(merged_data['CompetitionDistance'].median(), inplace=True)
merged_data['CompetitionOpenSinceMonth'].fillna(0, inplace=True)
merged_data['CompetitionOpenSinceYear'].fillna(0, inplace=True)

# Fill missing promo2 data with default values
merged_data['Promo2SinceWeek'].fillna(0, inplace=True)
merged_data['Promo2SinceYear'].fillna(0, inplace=True)
merged_data['PromoInterval'].fillna('None', inplace=True)

# Step 2: Feature engineering
# Convert date column to datetime
merged_data['Date'] = pd.to_datetime(merged_data['Date'])

# Extract temporal features from the date
merged_data['Year'] = merged_data['Date'].dt.year
merged_data['Month'] = merged_data['Date'].dt.month
merged_data['Day'] = merged_data['Date'].dt.day
merged_data['WeekOfYear'] = merged_data['Date'].dt.isocalendar().week
merged_data['DayOfYear'] = merged_data['Date'].dt.dayofyear
merged_data['DayOfWeek'] = merged_data['Date'].dt.dayofweek

# Beginning, mid, or end of the month
def month_stage(day):
    if day <= 10:
        return 'beginning'
    elif day <= 20:
        return 'mid'
    else:
        return 'end'

merged_data['MonthStage'] = merged_data['Day'].apply(month_stage)

# Compute competition age in months
merged_data['CompetitionAgeMonths'] = ((merged_data['Year'] - merged_data['CompetitionOpenSinceYear']) * 12 +
                                       (merged_data['Month'] - merged_data['CompetitionOpenSinceMonth']))
merged_data['CompetitionAgeMonths'] = merged_data['CompetitionAgeMonths'].apply(lambda x: max(x, 0))

# Step 3: Encode categorical variables

# Encode 'MonthStage' as numerical
month_stage_mapping = {'beginning': 1, 'mid': 2, 'end': 3}
merged_data['MonthStage'] = merged_data['MonthStage'].map(month_stage_mapping)
# Encode 'StateHoliday' as numerical
merged_data['StateHoliday'] = merged_data['StateHoliday'].replace({0: '0', '0': '0'})  # Ensure consistency
label_encoders = {}
for col in ['StoreType', 'Assortment', 'PromoInterval', 'StateHoliday']:
    label_enc = LabelEncoder()
    merged_data[col] = label_enc.fit_transform(merged_data[col])
    label_encoders[col] = label_enc  # Save encoders for later use



# Step 5: Check for any remaining missing values
if merged_data.isnull().sum().any():
    print("Warning: Missing values detected!")
    print(merged_data.isnull().sum())
else:
    print("No missing values remain!")

# Save the preprocessed dataset (optional)
merged_data.to_csv('preprocessed_data.csv', index=False)

# Display the first few rows of the preprocessed dataset
print(merged_data.head())


<ipython-input-13-f32e1f168849>:2: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_data = pd.read_csv('merged_data.csv')
<ipython-input-13-f32e1f168849>:18: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged_data['CompetitionDistance'].fillna(merged_data['CompetitionDistance'].median(), inplace=True)
<ipython-input-13-f32e1f168849>:19: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
T

Invalid dates found and removed:
        Store  DayOfWeek Date  Sales  Customers  Open  Promo StateHoliday  \
196219   1095          5  NaT    NaN        NaN   NaN    NaN          NaN   

        SchoolHoliday StoreType Assortment  CompetitionDistance  \
196219            NaN         a          a                690.0   

        CompetitionOpenSinceMonth  CompetitionOpenSinceYear  Promo2  \
196219                        6.0                    2007.0       1   

        Promo2SinceWeek  Promo2SinceYear    PromoInterval  
196219             14.0           2011.0  Jan,Apr,Jul,Oct  
No missing values remain!
   Store  DayOfWeek       Date    Sales  Customers  Open  Promo  StateHoliday  \
0      1          4 2015-07-31   5263.0      555.0   1.0    1.0             0   
1      2          4 2015-07-31   6064.0      625.0   1.0    1.0             0   
2      3          4 2015-07-31   8314.0      821.0   1.0    1.0             0   
3      4          4 2015-07-31  13995.0     1498.0   1.0    1.0 

# Scaling

# Model building with sklearn pipelines

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [17]:
# Load the preprocessed dataset
data = pd.read_csv('preprocessed_data.csv')

# Step 1: Define features (X) and target (y)
X = data.drop(['Sales', 'Date'], axis=1)  # Features
y = data['Sales']  # Target variable

# Step 2: Split the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 3: Define the pipeline
pipeline = Pipeline([
    ('model', RandomForestRegressor(random_state=42, n_estimators=100))  # Random Forest
])

# Step 4: Train the model
pipeline.fit(X_train, y_train)

# Step 5: Make predictions
y_pred = pipeline.predict(X_test)

# Step 6: Evaluate the model
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Model Evaluation:")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"R-squared (R2 Score): {r2:.4f}")

# Optional: Save the pipeline for deployment
import joblib
joblib.dump(pipeline, 'random_forest_pipeline.pkl')


Model Evaluation:
Mean Squared Error (MSE): 246298.6746
R-squared (R2 Score): 0.9833


['random_forest_pipeline.pkl']